# 25_이론 - OX 분류

> ## OX 손글씨 > LeNET 으로 분류하는 과제

In [1]:
from glob import glob
    # glob : Return a list of paths matching a pathname pattern.
    

In [2]:
train_raw_O_list = glob('/train_raw/O/*')

In [3]:
train_raw_O_list

[]

# skimage (scikit-image)

In [6]:
# !pip install scikit-image

In [7]:
from skimage.transform import rescale, resize
from skimage import color
from skimage.io import imread, imsave
import matplotlib.pyplot as plt

In [8]:
image = imread(train_raw_O_list[0])
image = color.rgb2gray(image)
plt.imshow(image,cmap='gray')

IndexError: list index out of range

In [9]:
# resize
img_resized = resize(image, (28, 28))
print(img_resized.shape)
plt.imshow(img_resized, cmap='gray')

NameError: name 'image' is not defined

In [ ]:
# 저장하기
import numpy as np
imsave('./tmp.png', np.round(img_resized*255).astype(np.uint8))

In [10]:
# 다시확인
tmp = imread('./tmp.png')
print(tmp.shape)
plt.imshow(tmp, cmap='gray');

FileNotFoundError: No such file: '/Users/newrandom/Documents/GitHub/DeepLearning_workspace/Zerobase_13_Deep Learning/tmp.png'

In [ ]:
# 파일 이름 추출 연습
train_raw_O_list[0].split('/')[-1][:-4]
                        # 폴더경로의 / 분리
                                # -1 : 맨 마지막, 즉 img_xxxx.jpg
                                    # [:-4] : 맨 마지막에서 4번째까지 자름 >> img_xxxx 만 남음

In [ ]:
# 이미지 resize 함수 만들기
def img_resize(img):
    img = color.rgb2gray(img)
    return resize(img, (28,28))
                        # color 이미지로 변환은 (28, 28, 3) 으로 해야한다.
                        # grayscale 이미지로의 변환은 (28, 28) 로만 하면된다.

In [ ]:
# Train_O
from tqdm.notebook import tqdm

def convert_train_O():
    train_raw_O_list = glob('./train_raw/O/*')
    for each in tqdm(train_raw_O_list):
        img = imread(each)
        img_resized = img_resize(img)
        save_name = './train/O/' + each.split('/')[-1][:-4] + '.png'
                        # jpg 파일을 png 파일로 변환하는 알고리즘.
        imsave(save_name, np.round(img_resized*255).astype(np.uint8))

convert_train_O()

In [ ]:
# Train_X
def convert_train_X():
    train_raw_X_list = glob('./train_raw/X/*')
    for each in tqdm(train_raw_X_list):
        img = imread(each)
        img_resized = img_resize(img)
        save_name = './train/X/' + each.split('/')[-1][:-4] + '.png'
                    # train_X 데이터의 저장 경로 지정
        imsave(save_name, np.round(img_resized*255).astype(np.uint8))

convert_train_X

In [ ]:
# Test_O
def convert_test_O():
    test_raw_O_list = glob('./test_raw/O/*')
    for each in tqdm(test_raw_O_list):
        img = imread(each)
        img_resized = img_resize(img)
        save_name = './test/O/' + each.split('/')[-1][:-4] + '.png'
        imsave(save_name, np.round(img_resized*255).astype(np.uint8))

convert_test_O

In [ ]:
# Test_X

def convert_test_X():
    test_raw_X_list = glob('./test_raw/X/*')
    for each in tqdm(test_raw_X_list):
        img = imread(each)
        img_resized = img_resize(img)
        save_name = './test/X/' + each.split('/')[-1][:-4] + '.png'
        imsave(save_name, np.round(img_resized * 255).astype(np.uint8))

convert_test_X()

# module import

In [11]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator    ##
import keras

np.random.seed(13)

# image_generator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    './train',
    target_size=(28, 28),
    batch_size = 3,
    class_mode = 'categorical')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    './test',
    target_size = (28, 28),
    batch_size = 3,
    class_mode = 'categorical'
)

In [ ]:
# model 구성
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(28,28,3)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))
                # 2 : O, X 로 정답을 가릴 것이기 때문에 2개
                            # softmax : 분류문제이기 때문에 softmax  사용

In [ ]:
# compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# fit
hist = model.fit_generator(
                # ImageDataGenerator 의 fit 은 fit_generator 를 사용한다.
    train_generator,
    steps_per_epoch=15,
    epochs=50, 
    validation_data = test_generator,
    validation_steps=5
)

In [12]:
plt.figure(figsize=(12,6))
plt.plot(hist.history['loss'], label = 'loss')
plt.plot(hist.history['val_loss'], label = 'val_loss')
plt.plot(hist.history['accuracy'], label = 'acc')
plt.plot(hist.history['val_accuracy'], label = 'val_acc')
plt.legend()
plt.show()

NameError: name 'hist' is not defined

<Figure size 864x432 with 0 Axes>

In [ ]:
scores = model.evaluate(
    test_generator,
    steps=5)

print('%s : %.2f%%' %(model.metrics_names[1], scores[1]*100))

In [ ]:
# predict
model.predict(test_generator)

In [ ]:
n = 1

def show_prediction_result(n):
    img = imread(test_generator.filepaths[n])
    pred = model.predict(np.expand_dims(color.gray2rgb(img), axis=0))
                        # np.expand_dims : Expand the shape of an array.
                                        # a : Input array.
                                                            # axis : Position in the expanded axes where the new axis (or axes) is placed.
    title = 'Predict : ' + str(np.argmax(pred))
                                # np.argmax() : 주어진 NumPy 배열에서 가장 높은 값을 가진 값의 인덱스를 반환합니다.
    plt.imshow(img/255., cmap='gray')
    plt.title(title)
    plt.show()

show_prediction_result(n)

In [ ]:
show_prediction_result(40)